# Mod 4 Project - Starter Notebook

This notebook has been provided to you so that you can make use of the following starter code to help with the trickier parts of preprocessing the Zillow dataset. 

The notebook contains a rough outline the general order you'll likely want to take in this project. You'll notice that most of the areas are left blank. This is so that it's more obvious exactly when you should make use of the starter code provided for preprocessing. 

**_NOTE:_** The number of empty cells are not meant to infer how much or how little code should be involved in any given step--we've just provided a few for your convenience. Add, delete, and change things around in this notebook as needed!

# Some Notes Before Starting

This project will be one of the more challenging projects you complete in this program. This is because working with Time Series data is a bit different than working with regular datasets. In order to make this a bit less frustrating and help you understand what you need to do (and when you need to do it), we'll quickly review the dataset formats that you'll encounter in this project. 

## Wide Format vs Long Format

If you take a look at the format of the data in `zillow_data.csv`, you'll notice that the actual Time Series values are stored as separate columns. Here's a sample: 

<img src='https://raw.githubusercontent.com/learn-co-students/dsc-mod-4-project-seattle-ds-102819/master/images/df_head.png'>

You'll notice that the first seven columns look like any other dataset you're used to working with. However, column 8 refers to the median housing sales values for April 1996, column 9 for May 1996, and so on. This This is called **_Wide Format_**, and it makes the dataframe intuitive and easy to read. However, there are problems with this format when it comes to actually learning from the data, because the data only makes sense if you know the name of the column that the data can be found it. Since column names are metadata, our algorithms will miss out on what dates each value is for. This means that before we pass this data to our ARIMA model, we'll need to reshape our dataset to **_Long Format_**. Reshaped into long format, the dataframe above would now look like:

<img src='https://raw.githubusercontent.com/learn-co-students/dsc-mod-4-project-seattle-ds-102819/master/images/melted1.png'>

There are now many more rows in this dataset--one for each unique time and zipcode combination in the data! Once our dataset is in this format, we'll be able to train an ARIMA model on it. The method used to convert from Wide to Long is `pd.melt()`, and it is common to refer to our dataset as 'melted' after the transition to denote that it is in long format. 

# Helper Functions Provided

Melting a dataset can be tricky if you've never done it before, so you'll see that we have provided a sample function, `melt_data()`, to help you with this step below. Also provided is:

* `get_datetimes()`, a function to deal with converting the column values for datetimes as a pandas series of datetime objects
* Some good parameters for matplotlib to help make your visualizations more readable. 

Good luck!


# Step 1: Load the Data/Filtering for Chosen Zipcodes

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv("zillow_data.csv")

In [4]:
df

,RegionID,RegionName,City,State,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,...,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04
0,84654,60657,Chicago,IL,Chicago,Cook,1,334200.0,335400.0,336500.0,...,1005500,1007500,1007800,1009600,1013300,1018700,1024400,1030700,1033800,1030600
1,90668,75070,McKinney,TX,Dallas-Fort Worth,Collin,2,235700.0,236900.0,236700.0,...,308000,310000,312500,314100,315000,316600,318100,319600,321100,321800
2,91982,77494,Katy,TX,Houston,Harris,3,210400.0,212200.0,212200.0,...,321000,320600,320200,320400,320800,321200,321200,323000,326900,329900
3,84616,60614,Chicago,IL,Chicago,Cook,4,498100.0,500900.0,503100.0,...,1289800,1287700,1287400,1291500,1296600,1299000,1302700,1306400,1308500,1307000
4,93144,79936,El Paso,TX,El Paso,El Paso,5,77300.0,77300.0,77300.0,...,119100,119400,120000,120300,120300,120300,120300,120500,121000,121500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14718,58333,1338,Ashfield,MA,Greenfield Town,Franklin,14719,94600.0,94300.0,94000.0,...,216800,217700,218600,218500,218100,216400,213100,209800,209200,209300
14719,59107,3293,Woodstock,NH,Claremont,Grafton,14720,92700.0,92500.0,92400.0,...,202100,208400,212200,215200,214300,213100,213700,218300,222700,225800
14720,75672,40404,Berea,KY,Richmond,Madison,14721,57100.0,57300.0,57500.0,...,121800,122800,124600,126700,128800,130600,131700,132500,133000,133400
14721,93733,81225,Mount Crested Butte,CO,NaN,Gunnison,14722,191100.0,192400.0,193700.0,...,662800,671200,682400,695600,695500,694700,706400,705300,681500,664400


In [19]:
for col in df.columns:
    print(col, df[col].isna().sum())

RegionID 0
RegionName 0
City 0
State 0
Metro 1043
CountyName 0
SizeRank 0
1996-04 1039
1996-05 1039
1996-06 1039
1996-07 1039
1996-08 1039
1996-09 1039
1996-10 1039
1996-11 1039
1996-12 1039
1997-01 1039
1997-02 1039
1997-03 1039
1997-04 1039
1997-05 1039
1997-06 1039
1997-07 1038
1997-08 1038
1997-09 1038
1997-10 1038
1997-11 1038
1997-12 1038
1998-01 1036
1998-02 1036
1998-03 1036
1998-04 1036
1998-05 1036
1998-06 1036
1998-07 1036
1998-08 1036
1998-09 1036
1998-10 1036
1998-11 1036
1998-12 1036
1999-01 1036
1999-02 1036
1999-03 1036
1999-04 1036
1999-05 1036
1999-06 1036
1999-07 1036
1999-08 1036
1999-09 1036
1999-10 1036
1999-11 1036
1999-12 1036
2000-01 1036
2000-02 1036
2000-03 1036
2000-04 1036
2000-05 1036
2000-06 1036
2000-07 1036
2000-08 1036
2000-09 1036
2000-10 1036
2000-11 1036
2000-12 1036
2001-01 1036
2001-02 1036
2001-03 1036
2001-04 1036
2001-05 1036
2001-06 1036
2001-07 1036
2001-08 1036
2001-09 1036
2001-10 1036
2001-11 1036
2001-12 1036
2002-01 1036
2002-02 1036
200

In [120]:
df[df.iloc[:,220].isna()]

,RegionID,RegionName,City,State,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,...,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04
2946,73623,35810,Huntsville,AL,Huntsville,Madison,2947,NaN,NaN,NaN,...,61000,61000,61100,61900,62800,63200,63700,65200,67500,69400
3330,58630,2116,Boston,MA,Boston,Suffolk,3331,NaN,NaN,NaN,...,1931100,1995600,2031100,2049600,2057800,2064500,2037600,2056900,2106500,2136500
6153,73629,35816,Huntsville,AL,Huntsville,Madison,6154,NaN,NaN,NaN,...,61500,62500,62800,63300,64100,65100,66600,68700,70400,71300
7587,78091,46320,Hammond,IN,Chicago,Lake,7588,NaN,NaN,NaN,...,66000,67900,68200,68700,69600,69400,68500,68500,68200,67200
7635,78566,47371,Portland,IN,NaN,Jay,7636,NaN,NaN,NaN,...,86300,86600,87400,88200,89100,90300,90400,89100,88000,88100
8263,88723,70647,Iowa,LA,Lake Charles,Calcasieu,8264,NaN,NaN,NaN,...,111600,117900,125300,128900,129600,130800,132100,132700,132400,132300
8338,73630,35824,Huntsville,AL,Huntsville,Madison,8339,NaN,NaN,NaN,...,217200,216800,216300,215100,215700,218200,222000,226200,230300,231800
8668,75206,39202,Jackson,MS,Jackson,Hinds,8669,NaN,NaN,NaN,...,152700,154300,155800,157500,158100,158300,158400,157900,156600,155300
8746,90561,74857,Norman,OK,Oklahoma City,Cleveland,8747,NaN,NaN,NaN,...,141800,141800,141700,141700,142000,143600,145600,145700,143900,141900
8780,78097,46327,Hammond,IN,Chicago,Lake,8781,NaN,NaN,NaN,...,75200,74600,73200,72500,72800,73200,72700,72900,74000,74400


In [102]:
temp = df.isna().sum()>0
temp = temp.loc[temp].index
for t in temp:
    

Index(['Metro', '1996-04', '1996-05', '1996-06', '1996-07', '1996-08',
       '1996-09', '1996-10', '1996-11', '1996-12',
       ...
       '2013-09', '2013-10', '2013-11', '2013-12', '2014-01', '2014-02',
       '2014-03', '2014-04', '2014-05', '2014-06'],
      dtype='object', length=220)

In [103]:
df[df.Metro.isna()]

,RegionID,RegionName,City,State,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,...,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04
151,69340,27410,Greensboro,NC,NaN,Guilford,152,137100.0,136600.0,136000.0,...,212900,213200,213600,214300,215100,216200,217700,219600,221000,221500
167,69336,27406,Greensboro,NC,NaN,Guilford,168,80900.0,80700.0,80400.0,...,109700,107900,106700,107200,107900,108700,109500,110000,110700,111300
491,69270,27265,High Point,NC,NaN,Guilford,492,108000.0,107700.0,107400.0,...,159300,157400,156300,156600,156700,156600,156800,157000,157400,157800
526,69337,27407,Greensboro,NC,NaN,Guilford,527,93900.0,93700.0,93400.0,...,122000,120800,119800,119800,120300,121100,122200,123100,124000,124900
695,69335,27405,Greensboro,NC,NaN,Guilford,696,74100.0,73900.0,73800.0,...,91500,90200,89000,88700,88900,89700,91000,92100,93100,93700
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14684,70215,28775,Scaly Mountain,NC,NaN,Macon,14685,71800.0,71900.0,72000.0,...,215900,215600,217400,219700,218900,215100,211400,213000,216500,216700
14694,99114,97149,Neskowin,OR,NaN,Tillamook,14695,187900.0,188900.0,189700.0,...,394800,399000,402900,408800,418900,428100,433000,431600,421600,413400
14710,59210,3812,Bartlett,NH,NaN,Carroll,14711,80900.0,80800.0,80800.0,...,215500,217000,219200,221700,223600,224800,226000,226900,227700,228000
14717,62697,12720,Bethel,NY,NaN,Sullivan,14718,62500.0,62600.0,62700.0,...,122200,122700,122300,122000,122200,122800,123200,123200,120700,117700


In [15]:
for col in df.columns:
    print(df[col].value_counts(dropna=False))

73724     1
70551     1
99221     1
76688     1
82829     1
         ..
71176     1
91654     1
65029     1
100380    1
98304     1
Name: RegionID, Length: 14723, dtype: int64
55324    1
74561    1
73538    1
31546    1
82070    1
        ..
75182    1
55343    1
1450     1
73129    1
65536    1
Name: RegionName, Length: 14723, dtype: int64
New York        114
Los Angeles      95
Houston          88
San Antonio      48
Washington       45
               ... 
Mount Dora        1
Robertsville      1
Grottoes          1
Tullahoma         1
Palm City         1
Name: City, Length: 7554, dtype: int64
CA    1224
NY    1015
TX     989
PA     831
FL     785
OH     588
IL     547
NJ     502
MI     499
IN     428
NC     428
MA     417
TN     404
VA     401
MN     375
GA     345
WA     341
WI     332
MO     319
MD     317
CO     249
KS     241
AZ     230
OR     224
OK     221
SC     206
NH     199
LA     193
AL     183
IA     158
MS     153
KY     139
CT     124
UT     121
ID     110
AR     105
NV

Name: 2001-02, Length: 3487, dtype: int64
NaN          1036
81300.0        19
82900.0        19
88600.0        18
150500.0       18
             ... 
257900.0        1
1104800.0       1
275400.0        1
1454100.0       1
282500.0        1
Name: 2001-03, Length: 3488, dtype: int64
NaN         1036
95200.0       21
81000.0       19
89100.0       18
122200.0      18
            ... 
234600.0       1
309400.0       1
289900.0       1
525000.0       1
416500.0       1
Name: 2001-04, Length: 3511, dtype: int64
NaN         1036
96500.0       20
95600.0       19
138700.0      18
109900.0      18
            ... 
33300.0        1
370900.0       1
332800.0       1
246000.0       1
227300.0       1
Name: 2001-05, Length: 3530, dtype: int64
NaN          1036
92200.0        22
97300.0        19
77600.0        18
123400.0       17
             ... 
406500.0        1
656900.0        1
398300.0        1
1636000.0       1
964800.0        1
Name: 2001-06, Length: 3528, dtype: int64
NaN         1036
101

Name: 2006-10, Length: 5163, dtype: int64
NaN         640
131100.0     14
85600.0      14
129000.0     14
86500.0      13
           ... 
682800.0      1
382500.0      1
782400.0      1
489000.0      1
319600.0      1
Name: 2006-11, Length: 5186, dtype: int64
NaN         640
132000.0     15
86800.0      15
140400.0     14
141500.0     13
           ... 
470200.0      1
678600.0      1
243500.0      1
315100.0      1
268800.0      1
Name: 2006-12, Length: 5160, dtype: int64
NaN          620
93600.0       14
81100.0       14
130200.0      14
142400.0      13
            ... 
809800.0       1
621000.0       1
898700.0       1
695100.0       1
1912900.0      1
Name: 2007-01, Length: 5143, dtype: int64
NaN         620
109500.0     15
123000.0     14
159800.0     13
101700.0     13
           ... 
329400.0      1
762400.0      1
619100.0      1
922100.0      1
207300.0      1
Name: 2007-02, Length: 5172, dtype: int64
NaN         620
99500.0      14
160100.0     13
130400.0     13
110000.0   

NaN         275
81600.0      20
127200.0     19
131600.0     16
87100.0      16
           ... 
336800.0      1
503800.0      1
349800.0      1
389100.0      1
372700.0      1
Name: 2011-03, Length: 4274, dtype: int64
NaN         275
80700.0      20
116300.0     16
123800.0     16
127200.0     16
           ... 
442400.0      1
476600.0      1
700000.0      1
390800.0      1
425200.0      1
Name: 2011-04, Length: 4268, dtype: int64
NaN         275
88500.0      20
118500.0     17
113700.0     16
113600.0     16
           ... 
307200.0      1
449400.0      1
296000.0      1
45900.0       1
203300.0      1
Name: 2011-05, Length: 4240, dtype: int64
NaN         275
105400.0     18
96400.0      17
112300.0     16
134300.0     16
           ... 
927500.0      1
308100.0      1
239300.0      1
873300.0      1
279000.0      1
Name: 2011-06, Length: 4256, dtype: int64
NaN          251
109800.0      17
105200.0      17
124700.0      16
128200.0      16
            ... 
637500.0       1
617400.0 

Name: 2016-10, Length: 5040, dtype: int64
162600    15
111400    15
181100    14
141500    14
129700    14
          ..
503800     1
634500     1
963700     1
627100     1
458700     1
Name: 2016-11, Length: 5052, dtype: int64
145600     14
145400     14
121400     14
129600     13
168600     13
           ..
418400      1
386800      1
2646600     1
630700      1
819200      1
Name: 2016-12, Length: 5079, dtype: int64
146100    16
169100    15
145400    14
122100    14
123700    14
          ..
437200     1
240600     1
300200     1
453600     1
409600     1
Name: 2017-01, Length: 5075, dtype: int64
129500     15
126300     14
125100     14
133300     14
85000      13
           ..
887500      1
1887500     1
283800      1
371700      1
616700      1
Name: 2017-02, Length: 5093, dtype: int64
156300     16
127300     15
150300     15
146500     14
115300     14
           ..
1082500     1
341300      1
339000      1
562000      1
1245300     1
Name: 2017-03, Length: 5135, dtype: int64


# Step 2: Data Preprocessing

In [ ]:
def get_datetimes(df):
    return pd.to_datetime(df.columns.values[1:], format='%Y-%m')

# Step 3: EDA and Visualization

In [ ]:
font = {'family' : 'normal',
        'weight' : 'bold',
        'size'   : 22}

matplotlib.rc('font', **font)

# NOTE: if you visualizations are too cluttered to read, try calling 'plt.gcf().autofmt_xdate()'!

# Step 4: Reshape from Wide to Long Format

In [2]:
def melt_data(df):
    melted = pd.melt(df, id_vars=['RegionName', 'City', 'State', 'Metro', 'CountyName'], var_name='time')
    melted['time'] = pd.to_datetime(melted['time'], infer_datetime_format=True)
    melted = melted.dropna(subset=['value'])
    return melted.groupby('time').aggregate({'value':'mean'})

In [11]:
df.columns

Index(['RegionID', 'RegionName', 'City', 'State', 'Metro', 'CountyName',
       'SizeRank', '1996-04', '1996-05', '1996-06',
       ...
       '2017-07', '2017-08', '2017-09', '2017-10', '2017-11', '2017-12',
       '2018-01', '2018-02', '2018-03', '2018-04'],
      dtype='object', length=272)

In [15]:
df.RegionID = df.RegionID.astype('str')

In [17]:
df.RegionID.dtype

dtype('O')

In [16]:
melted = melt_data(df)
melted

ValueError: ('Unknown string format:', 'RegionID')

# Step 5: ARIMA Modeling

# Step 6: Interpreting Results